## 0.5 - imports and read-ins

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import sklearn
import sklearn.decomposition as skdec
import matplotlib.pyplot as plt
import factor_analyzer
import random
from rpy2.robjects.packages import importr

from utils.clean_utils import transform_remove_skew, remove_outliers, remove_correlated_task_variables, drop_vars

from utils.r_to_py_utils import missForest

from utils.EFA_assumption_utils import check_sample_size, bartlett_sphericity, kmo

from utils.EFA_HCA_utils import Results

%load_ext rpy2.ipython

Using TensorFlow backend.


In [2]:
selected_variables = pd.read_csv('HDDM_meaningful_variables.csv', index_col='Unnamed: 0')

# 1 - Clean & impute the data

__selected_variables_clean = transform_remove_skew(selected_variables)__
 - for positively skewed (skew > 1) vars: 
  - shift so min is 0 (or 1?), then positive_subset = log(shifted(var)). remove outliers of positive subset.
  - keep those w/ new_skew < thresh, drop those are still too skewed
 - for negatively skewed (skew < 1) vars: 
  - negative subset = log(negative_subset.max()+1 - negative_subset) 

__selected_variables_clean = remove_outliers(selected_variables_clean)__
 - Removes outliers more than 1.5IQR below Q1 or above Q3
 
__selected_variables_clean = remove_correlated_task_variables(selected_variables_clean)__
 - Removes variables with corr > 0.85
   - removes both correlated vars?

__impute data__
- missForest

In [3]:
selected_variables_clean = transform_remove_skew(selected_variables)
selected_variables_clean = remove_outliers(selected_variables_clean)
selected_variables_clean = remove_correlated_task_variables(selected_variables_clean)

****************************************
** Successfully transformed 26 positively skewed variables:
adaptive_n_back.mean_load
angling_risk_task_always_sunny.release_coef_of_variation
bickel_titrator.hyp_discount_rate_large
bickel_titrator.hyp_discount_rate_medium
bickel_titrator.hyp_discount_rate_small
bis11_survey.Motor
choice_reaction_time.hddm_non_decision
columbia_card_task_cold.gain_sensitivity
columbia_card_task_hot.gain_sensitivity
directed_forgetting.proactive_interference_hddm_drift
dospert_eb_survey.health_safety
dot_pattern_expectancy.hddm_thresh
holt_laury_survey.beta
kirby.hyp_discount_rate_large
kirby.hyp_discount_rate_medium
kirby.hyp_discount_rate_small
motor_selective_stop_signal.hddm_thresh
shape_matching.hddm_thresh
shift_task.model_beta
simple_reaction_time.avg_rt
stim_selective_stop_signal.hddm_thresh
stop_signal.SSRT_high
stop_signal.hddm_thresh
stroop.hddm_thresh
threebytwo.hddm_thresh
tower_of_london.avg_move_time
****************************************
******

In [4]:
selected_variables_imputed, error = missForest(selected_variables_clean)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 3 in progress...done!


In [5]:
#### ONLY REQUIRED FOR THE SRO DATA - GRAB JUST TASK VARS

task_df = drop_vars(selected_variables_imputed, ['survey'], saved_vars = ['holt','cognitive_reflection'])
task_df_no_impute = drop_vars(selected_variables_clean, ['survey'], saved_vars = ['holt','cognitive_reflection'])

# 2 - Test assumptions for EFA

### 1. Sample size (participants:variables)

In [6]:
sample_ratio = check_sample_size(task_df)
# ncases, nvars = task_data.shape
# sample_ratio = ncases / nvars
# if sample_ratio > 3:
#     print('good enough for a pilot')
# if sample_ratio >= 5:
#     print('reasonable for a full analysis, but not ideal')
# if sample_ratio >= 20:
#     print('good enough to publish with!')

good enough for a pilot


### 2. bartlett test, want p < 0.5 (or 0.1, or 0.01 ...)

from https://www.statisticshowto.datasciencecentral.com/bartletts-test/#BTs :

Bartlett’s test for Sphericity compares your correlation matrix (a matrix of Pearson correlations) to the identity matrix. In other words, it checks if there is a redundancy between variables that can be summarized with some factors

In [7]:
chi_square_value,p=factor_analyzer.calculate_bartlett_sphericity(task_df)
chi_square_value, p

(32922.576204860205, 0.0)

In [8]:
chi_square_value1, p1 = ss.bartlett(*[task_df[col].values for col in task_df.columns])
print(chi_square_value1)
print(p1)
chi_square_value2, p2 = ss.bartlett(*task_df.values)
print(chi_square_value2)
print(p2)

756002.2468745925
0.0
12086.164338026812
0.0


In [9]:
chi2_spear,ddl_spear,pvalue_spear = bartlett_sphericity(task_df, corr_method="spearman")
print(chi2_spear,ddl_spear,pvalue_spear )
chi2_son,ddl_son,pvalue_son = bartlett_sphericity(task_df, corr_method="pearson")
print(chi2_son,ddl_son,pvalue_son)
print('pearson = same as the factor analyzer!')

33755.89865691212 8256.0 0.0
32804.5426116459 8256.0 0.0
pearson = same as the factor analyzer!


### 3. Kaiser-Meyer-Olkin (KMO) Test, want a score >= 0.6

from https://www.statisticshowto.datasciencecentral.com/kaiser-meyer-olkin/ :

Kaiser-Meyer-Olkin (KMO) Test is a measure of how suited your data is for Factor Analysis. The test measures sampling adequacy for each variable in the model and for the complete model. The statistic is a measure of the proportion of variance among variables that might be common variance. The lower the proportion, the more suited your data is to Factor Analysis.

KMO returns values between 0 and 1. A rule of thumb for interpreting the statistic:

KMO values between 0.8 and 1 indicate the sampling is adequate.

KMO values less than 0.6 indicate the sampling is not adequate and that remedial action should be taken. Some authors put this value at 0.5, so use your own judgment for values between 0.5 and 0.6.

KMO Values close to zero means that there are large partial correlations compared to the sum of correlations. In other words, there are widespread correlations which are a large problem for factor analysis

For reference, Kaiser put the following values on the results:

* 0.00 to 0.49 unacceptable.
* 0.50 to 0.59 miserable.
* 0.60 to 0.69 mediocre.
* 0.70 to 0.79 middling.
* 0.80 to 0.89 meritorious.
* 0.90 to 1.00 marvelous.

In [10]:
kmo_all,kmo_model=factor_analyzer.calculate_kmo(task_df)
kmo_model

/Users/henrymj/anaconda3/lib/python3.7/site-packages/factor_analyzer/utils.py:248: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn('The inverse of the variance-covariance matrix '


0.748513534160494

In [11]:
dataset_corr_spear = task_df.corr(method="spearman")
value_spear,per_variable_spear = kmo(dataset_corr_spear)
print(value_spear)

dataset_corr_son = task_df.corr(method="pearson")
value_son,per_variable_son = kmo(dataset_corr_son)
print(value_son)
print('pearson = same as the factor analyzer!')

0.7607458858956919
0.7492347745616416
pearson = same as the factor analyzer!


# 3 - Perform EFA

In [12]:
verbose=True
bootstrap=False

boot_iter = 1000 
ID = str(random.getrandbits(16)) 
print(ID)

48194


In [13]:
results = Results(data=task_df,
                  data_no_impute = task_df_no_impute,
                  dist_metric='abscorrelation',
                  name='task',
                  filter_regex='task',
                  boot_iter=boot_iter,
                  ID=ID)

In [14]:
for rotate in ['oblimin', 'varimax']:
    results.run_EFA_analysis(rotate=rotate, 
                             verbose=verbose, 
                             bootstrap=bootstrap)
    results.run_clustering_analysis(rotate=rotate, 
                                    verbose=verbose, 
                                    run_graphs=False)
#     c = results.EFA.get_c()
#     # name factors and clusters
#     factor_names = subset.get('%s_factor_names' % rotate, None)
#     cluster_names = subset.get('%s_cluster_names' % rotate, None)
#     if factor_names:
#         results.EFA.name_factors(factor_names, rotate=rotate)
#     if cluster_names:
#         results.HCA.name_clusters(cluster_names, inp='EFA%s_%s' % (c, rotate))

*******************************************************************************
Running EFA, rotate: oblimin
*******************************************************************************
Is the data adequate for factor analysis? Yes
Determining Optimal Dimensionality


R[write to console]: Loading required namespace: GPArotation



Best Components:  {'c_metric-BIC': 5}
Creating Factor Tree
No 5 factor solution computed yet! Computing...
Determining Higher Order Factors
# of components not specified, using BIC determined #
*******************************************************************************
Running HCA
*******************************************************************************
Clustering data


R[write to console]: Error in if (is.na(n) || n > 65536L) stop("size cannot be NA nor exceed 65536") : 
  missing value where TRUE/FALSE needed
Calls: <Anonymous>

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry,  :
R[write to console]: 
 
R[write to console]:  The response has five or fewer unique values.  Are you sure you want to do regression?

R[write to console]: 2: 
R[write to console]: In randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry,  :
R[write to console]: 
 
R[write to console]:  The response has five or fewer unique values.  Are you sure you want to do regression?

R[write to console]: 3: 
R[write to console]: In randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry,  :
R[write to console]: 
 
R[write to console]:  The response has five or fewer unique values.  Are you sure you want to do regressio

RRuntimeError: Error in if (is.na(n) || n > 65536L) stop("size cannot be NA nor exceed 65536") : 
  missing value where TRUE/FALSE needed
Calls: <Anonymous>


In [ ]:
cluster_EFA=True
run_graphs=False
rotate='oblimin'
verbose=True
dist_metric=None





In [ ]:
if verbose:
    print('*'*79)
    print('Running HCA')
    print('*'*79)
if dist_metric is None: 
    data=task_df
    self_EFA = results.EFA
    cluster_EFA = cluster_EFA
    rotate=rotate
    run_graphs=run_graphs
    verbose=verbose

In [ ]:
        if verbose: print("Clustering data")
        data=data
        dist_metric=None
        method='average'

In [ ]:
from utils.utils import distcorr

In [ ]:
        if dist_metric is None:
            dist_metric = distcorr
            label_append = ''
        df = data.T
        method=method
        pdist_kws={'metric': dist_metric}

In [ ]:
min_cluster_size=3
compute_dist=True
cluster_kws=None

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import leaves_list, linkage, cut_tree

In [ ]:
    # if compute_dist = False, assume df is a distance matrix. Otherwise
    # compute distance on df rows
    if compute_dist == True:
        if pdist_kws is None:
            pdist_kws= {'metric': 'correlation'}
        if pdist_kws['metric'] == 'abscorrelation':
            # convert to absolute correlations
            dist_vec = abs_pdist(df)
        elif pdist_kws['metric'] == 'sqcorrelation':
            # convert to squared correlations
            dist_vec = squareform(1-df.T.corr()**2)
        else:
            dist_vec = pdist(df, **pdist_kws)
        dist_df = pd.DataFrame(squareform(dist_vec), 
                               index=df.index, 
                               columns=df.index)
    else:
        assert df.shape[0] == df.shape[1]
        dist_df = df
        dist_vec = squareform(df.values)
    #clustering. This works the same as hclust
    link = linkage(dist_vec, method=method)    
    #dendrogram
    # same as order.dendrogram(as.dendrogram(hclust output)) in R
    reorder_vec = leaves_list(link)
    clustered_df = dist_df.iloc[reorder_vec, reorder_vec]
    # clustering
    if cluster_kws is None:
        cluster_kws = {'minClusterSize': 3,
                       'verbose': 0,
                       'pamStage': False}

In [ ]:
len(dist_vec)

In [ ]:
func='hybrid'
method=method
minClusterSize = 3
verbose = 0
pamStage = False

distance_df = dist_df

In [ ]:
    stats = importr('stats')
    dynamicTreeCut = importr('dynamicTreeCut')
    dist = stats.as_dist(distance_df)

In [ ]:
len(dist)

In [ ]:
type(dist)

In [ ]:
dist

In [ ]:
import rpy2
dist = rpy2.robjects.r['dist'](distance_df)

In [ ]:
len(dist)

In [ ]:
type(dist)

In [ ]:
np.max(dist)

In [ ]:
np.argwhere(np.isnan(dist))

In [ ]:
np.max(dist)

In [ ]:
    link = stats.hclust(dist.tolist(), method=method)


In [ ]:
type(stats.as_dist(distance_df))

In [ ]:
len(dist)

In [ ]:
clustered_df

In [ ]:
    def run_clustering_analysis(self, cluster_EFA=True, run_graphs=True,
                                rotate='oblimin',  dist_metric=None, verbose=False):
        """ Run HCA Analysis
        
        Args:
            dist_metric: if provided, create a new HCA instances with the
                provided dist_metric and return it. If None (default) run
                the results' internal HCA with the dist_metric provided
                at creation
        """
        if verbose:
            print('*'*79)
            print('Running HCA')
            print('*'*79)
        if dist_metric is None: 
            self.HCA.run(self.data, self.EFA, cluster_EFA=cluster_EFA,
                         rotate=rotate, run_graphs=run_graphs, verbose=verbose)

In [ ]:
    def run(self, data, EFA, cluster_EFA=False, rotate='oblimin',
            run_graphs=False, verbose=False):
        if verbose: print("Clustering data")
        self.cluster_data(data)
        if cluster_EFA:
            if verbose: print("Clustering EFA")
            self.cluster_EFA(EFA, EFA.get_c(),
                             rotate=rotate)
#         if run_graphs == True:
#             # run graph analysis on raw data
#             graphs = self.build_graphs('data', data)
#             self.results['data']['graphs'] = graphs

In [ ]:
    def cluster_data(self, data, dist_metric=None, method='average'):
        if dist_metric is None:
            dist_metric = self.dist_metric
            label_append = ''
        else:
            label_append = '_dist-%s' % dist_metric
        output = hierarchical_cluster(data.T, method=method,
                                      pdist_kws={'metric': dist_metric})
        self.results['data%s' % label_append] = output

In [ ]:
def hierarchical_cluster(df, compute_dist=True,  pdist_kws=None, 
                         method='average', min_cluster_size=3,
                         cluster_kws=None):
    """
    plot hierarchical clustering and heatmap
    :df: a correlation matrix
    parse_heatmap: int (optional). If defined, devides the columns of the 
                    heatmap based on cutting the dendrogram
    """
    
    # if compute_dist = False, assume df is a distance matrix. Otherwise
    # compute distance on df rows
    if compute_dist == True:
        if pdist_kws is None:
            pdist_kws= {'metric': 'correlation'}
        if pdist_kws['metric'] == 'abscorrelation':
            # convert to absolute correlations
            dist_vec = abs_pdist(df)
        elif pdist_kws['metric'] == 'sqcorrelation':
            # convert to squared correlations
            dist_vec = squareform(1-df.T.corr()**2)
        else:
            dist_vec = pdist(df, **pdist_kws)
        dist_df = pd.DataFrame(squareform(dist_vec), 
                               index=df.index, 
                               columns=df.index)
    else:
        assert df.shape[0] == df.shape[1]
        dist_df = df
        dist_vec = squareform(df.values)
    #clustering. This works the same as hclust
    link = linkage(dist_vec, method=method)    
    #dendrogram
    # same as order.dendrogram(as.dendrogram(hclust output)) in R
    reorder_vec = leaves_list(link)
    clustered_df = dist_df.iloc[reorder_vec, reorder_vec]
    # clustering
    if cluster_kws is None:
        cluster_kws = {'minClusterSize': 3,
                       'verbose': 0,
                       'pamStage': False}
    labels = dynamicTreeCut(dist_df, func='hybrid', method=method,  **cluster_kws)
    labels = reorder_labels(labels, link)
    return {'linkage': link, 
            'distance_df': dist_df, 
            'clustered_df': clustered_df,
            'reorder_vec': reorder_vec,
            'labels': labels}

In [ ]:
def dynamicTreeCut(distance_df, func='hybrid', method='average', **cluster_kws):
    """ uses DynamicTreeCut to find clusters
    Args:
        method = "hybrid" or "dyanmic":
    """
    stats = importr('stats')
    dynamicTreeCut = importr('dynamicTreeCut')
    dist = stats.as_dist(distance_df)
    link = stats.hclust(dist, method=method)
    if func == 'hybrid':
        dist = stats.as_dist(distance_df)
        clustering = dynamicTreeCut.cutreeHybrid(link, distance_df, **cluster_kws)
        return np.array(clustering[0])
    elif func == 'dynamic':
        clustering = dynamicTreeCut.cutreeDynamic(link, **cluster_kws)
        return np.array(clustering)